Возможные способы считывать аудио-файлы с помощью питона:

https://stackoverflow.com/questions/2060628/reading-wav-files-in-python

Stereo to Mono:

https://stackoverflow.com/questions/43056088/stereo-to-mono-wave-interpolation-in-python
https://stackoverflow.com/questions/30401042/stereo-to-mono-wav-in-python

Про считывание и записывание wave-файлов:

http://www.labbookpages.co.uk/audio/wavFiles.html

https://github.com/mbereket/music-transcription


Why is Fourier Transform so important?

https://dsp.stackexchange.com/questions/69/why-is-the-fourier-transform-so-important

Полезная презентация:

http://www.machinelearning.ru/wiki/images/c/cf/NizhibitskyMusicSlides.pdf


https://ru.wikipedia.org/wiki/Импульсно-кодовая_модуляция

## Data preparing 
### specifically for MAPS dataset

In [1]:
import soundfile as sf
import numpy as np
import os
import pandas as pd
# pd.set_option('display.max_rows', 20)

In [2]:
example_path = 'MAPS/AkPnStgb/MUS/MAPS_MUS-alb_esp2_AkPnStgb.wav'

In [3]:
data, samplerate = sf.read(example_path)

In [4]:
data.shape

(5620807, 2)

In [5]:
REL_PATH = 'MAPS/'

In [6]:
# checks if the files in the given triplet exists at all
def checkFileTripletValid(triplet):
    for filename in triplet:
        if os.path.isfile(filename):
            continue
        else:
            raise Exception("The given file doesn't exist")
    return True

In [7]:
files = []

for f in os.scandir(REL_PATH):
    if f.is_dir():
        for (dirpath, dirnames, filenames) in os.walk(f.path):
            for file in filenames:
                if file.endswith(".wav"):
                    fullpath = dirpath + '/' + file # forms the path to the file
                    base = os.path.splitext(fullpath)[0] # gets the path to the file without its extention
                    triplet = [base + '.wav', base + '.mid', base + '.txt'] # forms the triplet of files: .wav, .mid, .txt
                    if checkFileTripletValid(triplet):
                        files.append(triplet)


In [8]:
len(files)

29880

In [15]:
# transforms only 2-channel audiofile, others leaves untouched
def makeMonoAudiofile(data):
    if data.shape[1] == 2:
        data = (data[:, 0] + data[:,1]) / 2
    
def translateTimeToSampleNum(data):
    data["OnsetTime"] = np.round(data.values[:, 0] * SAMPLERATE).astype(int)
    data["OffsetTime"] = np.round(data.values[:, 1] * SAMPLERATE).astype(int)

A sample is the smallest usable quantum of digital audio. The term frame isn't formally defined in pure audio terms, but is often used in relation to video that may accompany an audio track. In that context a frame is the quantity of audio samples taken during a video frame interval.

In [10]:
FRAME_SIZE = 16384 # samples per frame
SAMPLERATE = 44100

# returns frames (numpy.ndarray of the shape (FRAME_SIZE, )) and labels for an audiofile
def preprocessAudiofile(path):
    
    # data (numpy.ndarray) – a two-dimensional NumPy array is returned,
    # where the channels are stored along the first dimension, i.e. as columns. 
    data, samplerate = sf.read(path)
    assert(samplerate==SAMPLERATE)
    makeMonoAudiofile(data)
    translateTimeToSampleNum(data)
    
    chunks_amount = len(data) // FRAME_SIZE
    samples_amount = FRAME_SIZE * chunks_amount
    # отбрасываем конец аудиофайла, если количество
    # сэмплов в нем НЕ делится ровно на FRAME_SIZE
    data = data[:samples_amount] 
    
    # делим data на одинаковые chunks_amount, если длина data
    # НЕ делится ровно на chunks_amount, то np.split должен будет выбросить exception
    frames = np.split(data, chunks_amount)
    
    
    return frames
    

In [ ]:
def get_label_for_

In [14]:
frames = preprocessAudiofile(example_path)
frames[1][16383]

0.00384521484375

According to my copy of the absolutely essential (!) Master Handbook of Acoustics, to hear shorter tones, that is sounds with a short impulse, they need to be louder:

A 1,000-Hz tone sounds like 1,000 Hz in a 1-second tone burst, but an extremely short burst sounds like a click. The duration of such a burst also influences the perceived loudness. Short bursts do not sound as loud as longer ones... A pulse 3 milliseconds long must have a level about 15dB higher to sound as loud as a 0.5-second (500 millisecond) pulse. Tones and random noise follow roughly the same relationship in loudness vs. pulse length.

The 100-msec region is significant... Only when the tones or noise bursts are shorter than this amount must the sound-pressure level be increased to produce a loudness equal to that of long pulses or steady tones or noise. This 100 msec appears to be the integrating time or the time constant of the human ear. (Everest 2001, 60-61)

In [152]:
data = pd.read_csv('MAPS/AkPnStgb/MUS/MAPS_MUS-alb_esp2_AkPnStgb.txt', sep="\t", header=0)
data

,OnsetTime,OffsetTime,MidiPitch
0,0.500004,2.45304,38
1,0.500004,2.45304,50
2,1.218850,2.45304,57
3,1.452060,2.45304,69
4,1.452060,2.45304,66
5,1.452060,2.45304,62
6,1.918480,2.45304,33
7,1.918480,2.45304,45
8,2.453040,4.16954,38
9,2.453040,4.16954,50


In [165]:
data["OnsetTime"] = np.round(data.values[:, 0] * SAMPLERATE).astype(int)
data["OffsetTime"] = np.round(data.values[:, 1] * SAMPLERATE).astype(int)

In [168]:
data

,OnsetTime,OffsetTime,MidiPitch
0,22050,108179,38
1,22050,108179,50
2,53751,108179,57
3,64036,108179,69
4,64036,108179,66
5,64036,108179,62
6,84605,108179,33
7,84605,108179,45
8,108179,183877,38
9,108179,183877,50
